In [1]:
import requests as rq
import lxml.html as lx
import pandas as pd
import re

In [2]:
#read html + make df for general stats


url = 'https://www.espn.com/nba/team/stats/_/name/sac/salary'
names = pd.read_html(url)[0]
stats = pd.read_html(url)[1]
kings_stats = pd.concat([names,stats],axis = "columns")

In [3]:
# read html + make df for shooting stats


shoot = pd.read_html(url)[3]
kings_shoot_stats = pd.concat([names,shoot], axis = "columns")

In [4]:
# clean up "name" and drop total from df


words = []
for i in kings_shoot_stats["Name"]:
    words.append(i.split()[0:2])
words = pd.DataFrame(words)
words.columns = ["First","Last"]
words["Names"] = words['First']+" "+ words["Last"]
words = words.drop("First", axis = 1).drop("Last", axis = 1)
kings_shoot_stats["Name"] = words
kings_shoot_stats = kings_shoot_stats.drop(16)

In [5]:
#create salary df
url2 = "https://www.espn.com/nba/team/roster/_/name/sac/salary"
kings_salary_raw = pd.read_html(url2)[0]

In [6]:
#Strips player number from each players name

for i in range(0,len(kings_salary_raw)):
    kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2060\3064642318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


In [7]:
#Cleans dataframe to only player name + salary
kings_salary = kings_salary_raw.set_index("Name").drop('Unnamed: 0', axis = 1)
kings_salary.columns
kings_salary = kings_salary.drop(['POS', 'Age', 'HT', 'WT', 'College'], axis = 1)

In [8]:
#combine dfs. final kings 2022 df


kings_stats_22 = kings_shoot_stats.join(kings_salary, on = "Name")

In [9]:
#create dict for 2018-2023 team stats


from time import sleep



tab = pd.DataFrame()
#Empty dictionary
tab_dict = {}
#Fetches data from 2018-2019 to 2022-2023 seasons
for i in range(2019, 2024):
    #Changes the url for each season
    url_loop = "https://www.espn.com/nba/stats/team/_/season/"+str(i)+"/seasontype/2"
    sleep(.02)
    #Gets dataframe with only team names
    team_names = pd.read_html(url_loop)[0]
    #Gets dataframe with all corresponding team stats
    team_stats = pd.read_html(url_loop)[1]
    #Concatenates the two above data frames
    tab = pd.concat([team_names,team_stats],axis = "columns")
    #Labels dataframe by year
    #tab = tab.style.set_caption(str(i-1)+"-"+str(i))
    name = "tab_"+str(i-1)
    #Adds to dictionary with 'tab_20xx' as key and the corresponding dataframe as the value
    tab_dict[name] = tab


In [10]:
#create dict for 2018-2023 opponent team stats


from time import sleep



tab = pd.DataFrame()
#Empty dictionary
tab_dict_op = {}
#Fetches data from 2018-2019 to 2022-2023 seasons
for i in range(2019, 2024):
    #Changes the url for each season
    url_loop_op = "https://www.espn.com/nba/stats/team/_/view/opponent/season/"+str(i)+"/seasontype/2"
    sleep(.02)
    #Gets dataframe with only team names
    team_names = pd.read_html(url_loop_op)[0]
    #Gets dataframe with all corresponding team stats
    team_stats = pd.read_html(url_loop_op)[1]
    #Concatenates the two above data frames
    tab = pd.concat([team_names,team_stats],axis = "columns")
    #Labels dataframe by year
    #tab = tab.style.set_caption(str(i-1)+"-"+str(i))
    name = "tab_op_"+str(i-1)
    #Adds to dictionary with 'tab_20xx' as key and the corresponding dataframe as the value
    tab_dict_op[name] = tab

tab_dict_op

{'tab_op_2018':     RK                    Team  GP    PTS   FGM   FGA   FG%   3PM   3PA   3P%  \
 0    1          Indiana Pacers  82  104.7  38.7  86.1  45.0  11.4  32.1  35.4   
 1    2              Miami Heat  82  105.9  38.3  86.8  44.1  11.9  33.1  35.8   
 2    3       Memphis Grizzlies  82  106.1  37.6  83.4  45.1  11.6  32.2  36.0   
 3    4               Utah Jazz  82  106.5  40.0  88.5  45.2   9.8  27.7  35.5   
 4    5           Orlando Magic  82  106.6  39.9  87.6  45.6  10.5  30.2  34.7   
 5    6          Denver Nuggets  82  106.7  39.6  86.4  45.8  10.8  31.9  33.9   
 6    7         Detroit Pistons  82  107.3  39.9  85.0  46.9   9.6  27.8  34.5   
 7    8          Boston Celtics  82  108.0  39.5  88.1  44.8  11.5  33.5  34.4   
 8    9         Toronto Raptors  82  108.4  40.3  89.7  44.9  10.8  31.2  34.5   
 9   10         Houston Rockets  82  109.1  40.4  86.7  46.6  10.1  29.8  34.0   
 10  11         Milwaukee Bucks  82  109.3  40.5  93.5  43.3  13.1  36.3  36.1   
 

In [11]:
nbateams = tab_dict['tab_2021']["Team"]
nbateams

0     Minnesota Timberwolves
1          Memphis Grizzlies
2            Milwaukee Bucks
3          Charlotte Hornets
4               Phoenix Suns
5              Atlanta Hawks
6                  Utah Jazz
7          San Antonio Spurs
8              Brooklyn Nets
9             Denver Nuggets
10        Los Angeles Lakers
11            Boston Celtics
12             Chicago Bulls
13            Indiana Pacers
14     Golden State Warriors
15          Sacramento Kings
16                Miami Heat
17        Philadelphia 76ers
18           Houston Rockets
19           Toronto Raptors
20      New Orleans Pelicans
21        Washington Wizards
22               LA Clippers
23          Dallas Mavericks
24       Cleveland Cavaliers
25           New York Knicks
26    Portland Trail Blazers
27           Detroit Pistons
28             Orlando Magic
29     Oklahoma City Thunder
Name: Team, dtype: object

team1 = names.columns[0]
names.columns = ["Team"]
new_row = pd.DataFrame({names.columns[0]:team1}, index=[0])
names = pd.concat([new_row,names.loc[:]]).reset_index(drop=True)
for i in range(0, len(names.iloc[:,0])):
    for team in nbateams:
        #print(team)
        if names.iloc[:,0][i].__contains__(team):
            names.iloc[:,0][i] = team
        else:
            continue
temp = names.join(stats.iloc[:,0:2])
temp
#names

def clean_teams(team, stats):
    team1 = team.columns[0]
    team.columns = ["Team"]
    new_row = pd.DataFrame({team.columns[0]:team1}, index=[0])
    team = pd.concat([new_row,team.loc[:]]).reset_index(drop=True)
    for i in range(0, len(team.iloc[:,0])):
        for teamname in nbateams:
            if team.iloc[:,0][i].__contains__(teamname):
                team.iloc[:,0][i] = teamname
            else:
                continue
    result = team.join(stats.iloc[:,0:2])
    return(result)

url4 = "https://www.espn.com/nba/standings/_/season/2022/group/league"
nbateams22 = pd.read_html(url4)[0]
nbastats22 = pd.read_html(url4)[1]
nbawins22 = clean_teams(nbateams, nbastats)
nbawins22

In [17]:
def clean_teams(team, stats):
    team1 = team.columns[0]
    team.columns = ["Team"]
    new_row = pd.DataFrame({team.columns[0]:team1}, index=[0])
    team = pd.concat([new_row,team.loc[:]]).reset_index(drop=True)
    for i in range(0, len(team.iloc[:,0])):
        for teamname in nbateams:
            if team.iloc[:,0][i].__contains__(teamname):
                team.iloc[:,0][i] = teamname
            else:
                continue
    result = team.join(stats.iloc[:,0:2])
    result["Made Playoffs"] = 0
    for index in result.index:
        if index < 10:
            result["Made Playoffs"][index] = 1
    return(result)

url3 = "https://www.espn.com/nba/standings/_/season/2023"
teams_east = pd.read_html(url3)[0]
stats_east = pd.read_html(url3)[1]
teams_west = pd.read_html(url3)[2]
stats_west = pd.read_html(url3)[3]
teams_east = clean_teams(teams_east, stats_east)
teams_west = clean_teams(teams_west, stats_west)
league_wins_losses = pd.concat([teams_east, teams_west], ignore_index = True)
league_wins_losses

C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2060\2836287342.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Made Playoffs"][index] = 1
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2060\2836287342.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Made Playoffs"][index] = 1


,Team,W,L,Made Playoffs
0,Boston Celtics,13,4,1
1,Milwaukee Bucks,12,4,1
2,Cleveland Cavaliers,11,6,1
3,Indiana Pacers,10,6,1
4,Washington Wizards,10,7,1
5,Atlanta Hawks,10,7,1
6,Toronto Raptors,9,8,1
7,New York Knicks,9,9,1
8,Philadelphia 76ers,8,8,1
9,Brooklyn Nets,8,9,1


In [18]:
url4 = "https://www.espn.com/nba/standings/_/season/2022"
teams_east = pd.read_html(url4)[0]
stats_east = pd.read_html(url4)[1]
teams_west = pd.read_html(url4)[2]
stats_west = pd.read_html(url4)[3]
teams_east = clean_teams(teams_east, stats_east)
teams_west = clean_teams(teams_west, stats_west)
league_wins_losses = pd.concat([teams_east, teams_west], ignore_index = True)
league_wins_losses

C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2060\2836287342.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Made Playoffs"][index] = 1
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2060\2836287342.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Made Playoffs"][index] = 1


,Team,W,L,Made Playoffs
0,Miami Heat,53,29,1
1,Boston Celtics,51,31,1
2,Milwaukee Bucks,51,31,1
3,Philadelphia 76ers,51,31,1
4,Toronto Raptors,48,34,1
5,Chicago Bulls,46,36,1
6,Brooklyn Nets,44,38,1
7,Atlanta Hawks,43,39,1
8,Cleveland Cavaliers,44,38,1
9,Charlotte Hornets,43,39,1


url4 = "https://www.espn.com/nba/standings/_/season/2022"

team1 = names.columns[0]
names.columns = ["Team"]
new_row = pd.DataFrame({names.columns[0]:team1}, index=[0])
names = pd.concat([new_row,names.loc[:]]).reset_index(drop=True)
names

names1 = pd.read_html(url4)[2]
stats1 = pd.read_html(url4)[3]
team1 = names1.columns[0]
names1.columns = ["Team"]
new_row = pd.DataFrame({names1.columns[0]:team1}, index=[0])
names1 = pd.concat([new_row,names1.loc[:]]).reset_index(drop=True)
names1

pd.concat([names, names1], ignore_index = True)

for i in range(0, len(names.iloc[:,0])):
    for team in nbateams:
        #print(team)
        if names.iloc[:,0][i].__contains__(team):
            names.iloc[:,0][i] = team
        else:
            continue
temp = names.join(stats.iloc[:,0:2])
temp

In [14]:
#Dataframes available:
kings_stats
kings_shoot_stats
kings_salary
tab_dict #tab_dict['tab_2019]...tab_dict['tab_2022']
tab_dict_op
kings_stats_22

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF,Salary
0,De'Aaron Fox,9.6,17.3,55.8,1.9,4.9,38.2,4.3,5.1,84.5,7.8,12.4,62.6,1.471,0.61,"$28,103,550"
1,Domantas Sabonis,6.1,10.4,59.0,0.3,1.1,31.3,4.9,6.6,74.7,5.8,9.3,62.1,1.686,0.61,"$18,500,000"
2,Kevin Huerter,5.7,11.5,50.0,3.7,7.4,49.5,1.5,2.2,66.7,2.1,4.1,50.8,1.448,0.66,"$14,508,929"
3,Malik Monk,4.4,9.7,45.2,1.9,5.3,36.7,2.0,2.1,93.8,2.5,4.5,55.2,1.308,0.55,"$9,472,219"
4,Harrison Barnes,3.9,8.3,46.4,0.9,3.3,26.5,4.1,4.8,84.7,3.0,5.1,59.2,1.520,0.52,"$18,352,273"
5,Keegan Murray,4.5,10.2,44.7,2.2,6.0,35.9,0.8,1.1,78.6,2.4,4.2,57.4,1.189,0.55,"$8,008,440"
6,Terence Davis,2.7,5.6,48.1,1.5,3.6,42.0,0.9,1.0,85.7,1.2,2.1,58.6,1.380,0.61,"$4,000,000"
7,Trey Lyles,2.3,5.8,39.5,1.5,4.2,34.5,0.8,1.2,66.7,0.8,1.6,52.4,1.171,0.52,"$2,700,000"
8,Davion Mitchell,2.7,5.8,47.1,0.9,2.8,33.3,0.3,0.4,66.7,1.8,3.0,60.0,1.149,0.55,"$4,833,600"
9,Chimezie Metu,2.2,3.9,55.8,0.2,1.2,15.4,0.5,0.6,85.7,2.0,2.7,73.3,1.302,0.58,"$1,910,680"
